In [1]:
%cd ..

import pandas as pd
pd.set_option('display.max_columns', None)

from scripts.utils_latest import *

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
df = pd.read_csv("results/7_results_of_first_4_rows.csv")
df.head()

,id,mrna_start,mrna_end,mrna_sequence,mirna_accession,mirna_start,mirna_end,mirna_sequence,mrna_dot_bracket_5to3,mirna_dot_bracket_5to3,pred_energy,is_mutated
0,1_809687_G_C_MIMAT0000062,5,26,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000062,0,17,TGAGGTAGTAGGTTGTATAGTT,.(((((.......((((.((.,)).))))....))))).,-6.9,0
1,1_809687_G_C_MIMAT0004481,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004481,0,21,CTATACAATCTACTGTCTTTC,.((((((..(((..((...(((((.,.)))))..)).))).)))))),-9.5,0
2,1_809687_G_C_MIMAT0010195,32,61,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0010195,0,22,CTGTACAGCCTCCTAGCTTTCC,.(((.(((((..(((((...((((.(.((,)).).))))))))))))))))),-16.8,0
3,1_809687_G_C_MIMAT0000063,15,57,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000063,4,22,TGAGGTAGTAGGTTGTGTGGTT,.(((..(((((((.......................(((((.,.))))))))))))..))),-8.8,0
4,1_809687_G_C_MIMAT0004482,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004482,0,22,CTATACAACCTACTGCCTTCCC,.(((((.(.(((...(((.(((((.,.)))))..)))))).)))))).,-15.0,0


In [3]:
def generate_positions_from_id(vcf_df):
    vcf_df['chr'] = vcf_df['id'].str.split('_').str[0]

    vcf_df['start_coordinate'] = vcf_df['id'].str.split('_').str[1].astype(int) - 30 + vcf_df["mrna_start"]
    vcf_df['end_coordinate'] = vcf_df['id'].str.split('_').str[1].astype(int) - 30 + vcf_df["mrna_end"]
    
    return vcf_df

df = generate_positions_from_id(df)
    


In [4]:
df = generate_alignment_string_from_dot_bracket(df)
df = generate_match_count_columns(df)


df.head()

,id,mrna_start,mrna_end,mrna_sequence,mirna_accession,mirna_start,mirna_end,mirna_sequence,mrna_dot_bracket_5to3,mirna_dot_bracket_5to3,pred_energy,is_mutated,chr,start_coordinate,end_coordinate,alignment_string,pred_num_basepairs,pred_seed_basepairs
0,1_809687_G_C_MIMAT0000062,5,26,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000062,0,17,TGAGGTAGTAGGTTGTATAGTT,.(((((.......((((.((.,)).))))....))))).,-6.9,0,1,809662,809683,110111100001111100000,11,5
1,1_809687_G_C_MIMAT0004481,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004481,0,21,CTATACAATCTACTGTCTTTC,.((((((..(((..((...(((((.,.)))))..)).))).)))))),-9.5,0,1,809686,809711,011111001101110111111,16,5
2,1_809687_G_C_MIMAT0010195,32,61,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0010195,0,22,CTGTACAGCCTCCTAGCTTTCC,.(((.(((((..(((((...((((.(.((,)).).))))))))))))))))),-16.8,0,1,809689,809718,1101011111111111111111,20,4
3,1_809687_G_C_MIMAT0000063,15,57,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000063,4,22,TGAGGTAGTAGGTTGTGTGGTT,.(((..(((((((.......................(((((.,.))))))))))))..))),-8.8,0,1,809672,809714,0000011111111111100111,15,2
4,1_809687_G_C_MIMAT0004482,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004482,0,22,CTATACAACCTACTGCCTTCCC,.(((((.(.(((...(((.(((((.,.)))))..)))))).)))))).,-15.0,0,1,809686,809711,0111110011111101111110,17,5


In [5]:
def generate_ta_sps_columns(df):
    # Generate temporary seed column
    df["seed"] = df["mirna_sequence"].str[1:8].str.replace("T", "U")
    # Read ta sps data
    ta_sps_df = pd.read_csv("data/processed/ta_sps/ta_sps.csv", usecols=["seed_8mer", "ta_log10", "sps_mean"])
    ta_sps_df = ta_sps_df.rename(columns={"seed_8mer": "seed"})
    # Merge dataframes on seed column
    df = df.merge(ta_sps_df, on="seed", how="left")
    # Drop temporary column
    df.drop(columns=["seed"], inplace=True)

    return df

df = generate_ta_sps_columns(df)

df.head()


,id,mrna_start,mrna_end,mrna_sequence,mirna_accession,mirna_start,mirna_end,mirna_sequence,mrna_dot_bracket_5to3,mirna_dot_bracket_5to3,pred_energy,is_mutated,chr,start_coordinate,end_coordinate,alignment_string,pred_num_basepairs,pred_seed_basepairs,ta_log10,sps_mean
0,1_809687_G_C_MIMAT0000062,5,26,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000062,0,17,TGAGGTAGTAGGTTGTATAGTT,.(((((.......((((.((.,)).))))....))))).,-6.9,0,1,809662,809683,110111100001111100000,11,5,3.393,-8.180
1,1_809687_G_C_MIMAT0004481,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004481,0,21,CTATACAATCTACTGTCTTTC,.((((((..(((..((...(((((.,.)))))..)).))).)))))),-9.5,0,1,809686,809711,011111001101110111111,16,5,3.716,-3.710
2,1_809687_G_C_MIMAT0010195,32,61,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0010195,0,22,CTGTACAGCCTCCTAGCTTTCC,.(((.(((((..(((((...((((.(.((,)).).))))))))))))))))),-16.8,0,1,809689,809718,1101011111111111111111,20,4,3.549,-6.275
3,1_809687_G_C_MIMAT0000063,15,57,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000063,4,22,TGAGGTAGTAGGTTGTGTGGTT,.(((..(((((((.......................(((((.,.))))))))))))..))),-8.8,0,1,809672,809714,0000011111111111100111,15,2,3.393,-8.180
4,1_809687_G_C_MIMAT0004482,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004482,0,22,CTATACAACCTACTGCCTTCCC,.(((((.(.(((...(((.(((((.,.)))))..)))))).)))))).,-15.0,0,1,809686,809711,0111110011111101111110,17,5,3.716,-3.710


# generating MRE sequence

In [6]:
def generate_mre_sequence_for_vcf(vcf_df):

    def slice_column(row):
        return row["mrna_sequence"][row["mre_start"]:row["mre_end"]]
    
    # getting mirna length
    vcf_df["mirna_length"] = vcf_df["mirna_sequence"].str.len()

    # using mirna length to figure out mre coordinates
    vcf_df["mre_end"] = vcf_df["mrna_end"] + vcf_df["mirna_start"]
    vcf_df["mre_start"] = vcf_df["mre_end"] - vcf_df["mirna_length"]

    # some start values might be lower than zero, so we need to adjust
    vcf_df["mre_start"] = vcf_df["mre_start"].apply(lambda x: max(x, 0))

    # creating mre sequence column
    vcf_df["mre_region"] = vcf_df.apply(slice_column, axis=1)

    # dropping temp column
    vcf_df.drop(columns=["mirna_length"], inplace=True)
    
    return vcf_df

df = generate_mre_sequence_for_vcf(df)

In [7]:
def generate_important_sites(df):
    df["anchor_a"] = (df["mre_region"].str[-1] == "A").astype(int)
    df["6mer_seed"] = (df["alignment_string"].str[1:7].str.count("0") == 0).astype(int)
    df["match_8"] = (df["alignment_string"].str[7] == "1").astype(int)
    df["6mer_seed_1_mismatch"] = (df["alignment_string"].str[1:7].str.count("0") == 1).astype(int)
    
    df["compensatory_site"] = (df["alignment_string"].str[12:17].str.count("0") == 0).astype(int)
    
    df["supplementary_site"] = (df["alignment_string"].str[12:16].str.count("0") == 0).astype(int)
    df["supplementary_site_2"] = (df["alignment_string"].str[16:21].str.count("0") == 0).astype(int)
    df["empty_seed"] = (df["alignment_string"].str[1:8].str.count("1") == 0).astype(int)
    
    
    df["9_consecutive_match_anywhere"] = (df["alignment_string"]
                                          .str
                                          .contains("1{" + str(9) + ",}")
                                          .astype(int))
    
    return df

df = generate_important_sites(df)


In [8]:
def generate_mirna_conservation_column(df):
    targetscan = (pd.read_csv("data/processed/mirbase/mirbase22.csv", 
                            usecols=["accession", "conservation"])
                            .rename(columns={"accession": "mirna_accession", "conservation": "mirna_conservation"})
                            [["mirna_accession", "mirna_conservation"]])

    df = df.merge(targetscan, on="mirna_accession", how="left")
    return df

df = generate_mirna_conservation_column(df)


In [9]:
def generate_seed_type_columns(df):
    df['seed_8mer'] = ((df['anchor_a'] == 1) & (df['6mer_seed'] == 1) & (df['match_8'] == 1)).astype(int)
    df['seed_7mer_a1'] = ((df['anchor_a'] == 1) & (df['6mer_seed'] == 1) & (df['match_8'] == 0)).astype(int)
    df['seed_7mer_m8'] = ((df['anchor_a'] == 0) & (df['6mer_seed'] == 1) & (df['match_8'] == 1) & (df['supplementary_site'] == 0) & (df['supplementary_site_2'] == 0)).astype(int)
    df['seed_compensatory'] = ((df['compensatory_site'] == 1) & (df['6mer_seed_1_mismatch'] == 1) & (df['match_8'] == 1)).astype(int)

    df['seed_clash_2'] = ((df['supplementary_site'] == 1) & (df['6mer_seed'] == 1) & (df['match_8'] == 1)).astype(int)
    df['seed_clash_3'] = ((df['supplementary_site_2'] == 1) & (df['6mer_seed'] == 1) & (df['match_8'] == 1)).astype(int)
    df['seed_clash_4'] = ((df['empty_seed'] == 1) & (df['9_consecutive_match_anywhere'] == 1)).astype(int)
    df['seed_clash_5'] = ((df['pred_num_basepairs'] > 10) & (df['6mer_seed'] == 0)).astype(int)
    
    return df


df = generate_seed_type_columns(df)


In [10]:
def generate_mre_au_content_column(df):
    
    def calculate_au_content(sequence):
        au_count = sequence.count('A') + sequence.count('T') + sequence.count('U')
        return None if len(sequence) == 0 else au_count / len(sequence)


    df["mre_au_content"] = df['mre_region'].apply(calculate_au_content)
    
    return df

df = generate_mre_au_content_column(df)


In [11]:
def generate_au_content_column_for_vcf(vcf_df):
    
    def calculate_au_content(sequence):
        au_count = sequence.count('A') + sequence.count('T') + sequence.count('U')
        return None if len(sequence) == 0 else au_count / len(sequence)

    vcf_df["au_content_sequence"] = vcf_df.apply(lambda x: get_nucleotides_in_interval(x['chr'], x['start_coordinate']-30, x["end_coordinate"]+30), axis=1)

    vcf_df["au_content"] = vcf_df['au_content_sequence'].apply(calculate_au_content)
    
    return vcf_df

df = generate_au_content_column_for_vcf(df)


In [12]:
df.head()

,id,mrna_start,mrna_end,mrna_sequence,mirna_accession,mirna_start,mirna_end,mirna_sequence,mrna_dot_bracket_5to3,mirna_dot_bracket_5to3,pred_energy,is_mutated,chr,start_coordinate,end_coordinate,alignment_string,pred_num_basepairs,pred_seed_basepairs,ta_log10,sps_mean,mre_end,mre_start,mre_region,anchor_a,6mer_seed,match_8,6mer_seed_1_mismatch,compensatory_site,supplementary_site,supplementary_site_2,empty_seed,9_consecutive_match_anywhere,mirna_conservation,seed_8mer,seed_7mer_a1,seed_7mer_m8,seed_compensatory,seed_clash_2,seed_clash_3,seed_clash_4,seed_clash_5,mre_au_content,au_content_sequence,au_content
0,1_809687_G_C_MIMAT0000062,5,26,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000062,0,17,TGAGGTAGTAGGTTGTATAGTT,.(((((.......((((.((.,)).))))....))))).,-6.9,0,1,809662,809683,110111100001111100000,11,5,3.393,-8.180,26,4,CAGCAATATCAGGATGCCACGG,0,0,0,1,0,1,0,0,0,2.0,0,0,0,0,0,0,0,1,0.454545,GCAACATGAACGACCTGGTGTCTGAATATCAGCAATATCAGGATGC...,0.500000
1,1_809687_G_C_MIMAT0004481,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004481,0,21,CTATACAATCTACTGTCTTTC,.((((((..(((..((...(((((.,.)))))..)).))).)))))),-9.5,0,1,809686,809711,011111001101110111111,16,5,3.716,-3.710,54,33,GGAGGAGTTTGAGGAGTATGC,0,0,0,1,0,0,1,0,0,-1.0,0,0,0,0,0,0,0,1,0.476190,AATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGA...,0.523256
2,1_809687_G_C_MIMAT0010195,32,61,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0010195,0,22,CTGTACAGCCTCCTAGCTTTCC,.(((.(((((..(((((...((((.(.((,)).).))))))))))))))))),-16.8,0,1,809689,809718,1101011111111111111111,20,4,3.549,-6.275,61,39,GTTTGAGGAGTATGCTGAGGAG,0,0,1,0,1,1,1,0,1,-1.0,0,0,0,0,0,0,0,1,0.500000,ATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAGGA...,0.511111
3,1_809687_G_C_MIMAT0000063,15,57,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0000063,4,22,TGAGGTAGTAGGTTGTGTGGTT,.(((..(((((((.......................(((((.,.))))))))))))..))),-8.8,0,1,809672,809714,0000011111111111100111,15,2,3.393,-8.180,61,39,GTTTGAGGAGTATGCTGAGGAG,0,0,1,0,1,1,0,0,1,2.0,0,0,0,0,0,0,0,1,0.500000,CGACCTGGTGTCTGAATATCAGCAATATCAGGATGCCACGGCTGAG...,0.504854
4,1_809687_G_C_MIMAT0004482,29,54,ATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGAG...,MIMAT0004482,0,22,CTATACAACCTACTGCCTTCCC,.(((((.(.(((...(((.(((((.,.)))))..)))))).)))))).,-15.0,0,1,809686,809711,0111110011111101111110,17,5,3.716,-3.710,54,32,AGGAGGAGTTTGAGGAGTATGC,0,0,0,1,0,0,1,0,0,-1.0,0,0,0,0,0,0,0,1,0.500000,AATATCAGCAATATCAGGATGCCACGGCTGAGGAGGAGGAGTTTGA...,0.523256


In [13]:
df.to_csv("results/8_adding_feature_cols.csv", index=False)